In [7]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

In [8]:
import tensorflow as tf

In [9]:
slim = tf.contrib.slim

In [10]:
def VGG_scope(weight_decay=0.0005):
    with slim.arg_scope([slim.conv2d,slim.fully_connected],
                       activation_fn = tf.nn.relu,
                       biases_initializer = tf.zeros_initializer(),
                       weights_regularizer=slim.l2_regularizer(weight_decay)):
        with slim.arg_scope([slim.conv2d],padding='SAME') as arg_sc:
            return arg_sc
        
def VGG_Net11(inputs,
            num_classes=1000,
            is_training=True,
            dropout_keep_prob=0.5,
            spatial_squeeze=True,
            scope='VGG_Net11'):
    with tf.variable_scope(scope, 'VGG_Net11',[inputs]) as sc:
        end_point_collection = sc.name + '_end_points'
        with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                           output_collections=[end_point_collections]):
            net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
            net = slim.max_pool2d(net, [2, 2], scope='pool1')
            net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
            net = slim.max_pool2d(net, [2, 2], scope='pool2')
            net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
            net = slim.max_pool2d(net, [2, 2], scope='pool3')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv4')
            net = slim.max_pool2d(net, [2, 2], scope='pool4')
            net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3], scope='conv5')
            net = slim.max_pool2d(net, [2, 2], scope='pool5')
            net = slim.conv2d(net, 4096, [7, 7], padding=fc_conv_padding, scope='fc6')
            net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                         scope='dropout6')
            net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
            net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                         scope='dropout7')
            net = slim.conv2d(net, num_classes, [1, 1],
                        activation_fn=None,
                        normalizer_fn=None,
                        scope='fc8')
            end_points = slim.utils.convert_collection_to_dict(end_points_collection)
            if spatial_squeeze:
                    net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
                    end_points[sc.name + '/fc8'] = net
                    return net, end_points
       
    alex_net.default_image_size=224        